In [1]:
import pandas as pd
from pyrosm import OSM, get_data
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
property_df = pd.read_csv("/Users/matthewrush/Documents/generic-real-estate-consulting-project-group-32/data/curated/pre_processed_data.csv", usecols=['url', 'postcode', 'school_duration', 'school_distance', 'park_duration', 'park_distance', 'shop_duration', 'shop_distance'])
postcode_df = pd.read_csv("/Users/matthewrush/Documents/generic-real-estate-consulting-project-group-32/data/raw/postcode.csv", usecols=['postcode', 'locality'])

/Users/matthewrush/virtualenv/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
property_df = property_df.groupby('postcode').mean().reset_index()

In [137]:
postcode_df[postcode_df['postcode'] == 3183]['locality'].values.tolist()

['BALACLAVA', 'ST KILDA EAST']

In [3]:
# Get test data
fp = get_data("melbourne")

osm = OSM(fp)

In [14]:
def park_count(osm, base_map, postcode, postcode_df):
    suburbs = postcode_df[postcode_df["postcode"] == int(postcode)][
        "locality"
    ].values.tolist()

    count = 0
    for i in suburbs:
        try:
            local_map = osm.get_boundaries(name=i.title())
            bbox_geom = local_map["geometry"].values[0]
            osm2 = OSM(base_map, bounding_box=bbox_geom)

            park_filter = {
                "leisure": [
                    "park",
                    "dog_park",
                    "garden",
                    "nature_reserve",
                    "park;playground",
                    "recreation_ground",
                    "reserve",
                ]
            }

            area = osm2.get_pois(custom_filter=park_filter)
            count = count + len(area["name"].unique())
        except Exception as e:
            print(f"{i}, {e}")
    print(postcode)
    return count


def shop_count(osm, base_map, postcode, postcode_df):
    suburbs = postcode_df[postcode_df["postcode"] == int(postcode)][
        "locality"
    ].values.tolist()

    count = 0
    for i in suburbs:
        try:
            local_map = osm.get_boundaries(name="Docklands")
            bbox_geom = local_map["geometry"].values[0]
            osm2 = OSM(base_map, bounding_box=bbox_geom)

            shop_filter = {
                "shop": [
                    "department_store",
                    "mall",
                    "garden",
                    "nature_reserve",
                    "park;playground",
                    "recreation_ground",
                    "reserve",
                ]
            }
            area = osm2.get_pois(custom_filter=shop_filter)
            count = count + len(area["name"].unique())
        except Exception as e:
            print(f"{i}, {e}")

    return count


In [8]:
test = property_df.iloc[0:4, ]

In [15]:
property_df['park_count'] = property_df.apply(lambda x: park_count(osm, fp, x['postcode'], postcode_df), axis=1)

/Users/matthewrush/virtualenv/lib/python3.10/site-packages/pyrosm/pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(


3000.0
3002.0


/Users/matthewrush/virtualenv/lib/python3.10/site-packages/pyrosm/pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(


3003.0


/Users/matthewrush/virtualenv/lib/python3.10/site-packages/pyrosm/pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(


...

3040.0


/Users/matthewrush/virtualenv/lib/python3.10/site-packages/pyrosm/pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(


3041.0
NIDDRIE NORTH, index 0 is out of bounds for axis 0 with size 0
3042.0
3043.0


/Users/matthewrush/virtualenv/lib/python3.10/site-packages/pyrosm/pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(
/Users/matthewrush/virtualenv/lib/python3.10/site-packages/pyrosm/pyrosm.py:576: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gdf = get_poi_data(


3044.0
3046.0
3047.0
3048.0


In [ ]:
property_df